### Library


In [1]:
import copy
import itertools as it
import networkx as nx
import numpy as np
import pandas as pd


### Data input

1. You can type the given table from the question to the DPTSP.csv directly without any convertion
2. OR You can type the converted table directly to the DPTSP.csv and delete the following line in the input panel
    - df = df.applymap(lambda x: int(userCode[x-1]) if x != 0 else x)


### Data convertion


In [2]:
df = pd.read_csv("DPTSP.csv", header=None, dtype=int)
df.index = (df.index + 1).astype(str)
df.columns = (df.columns + 1).astype(str)


### Input panel


In [3]:
userCode = "2694876301375324"
start = "5"  # normally should be 1


In [4]:
# Mapping link cost j to jth digit of the code
df = df.applymap(lambda x: int(userCode[x-1]) if x != 0 else x)
G = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())


### New Adjacency Matrix


In [5]:
print(df)


   1  2  3  4  5
1  0  9  2  8  4
2  2  0  8  4  9
3  8  4  0  6  2
4  9  2  9  0  9
5  8  6  4  2  0


### Optimal policy function by steps


In [6]:
def opf(it: int, nodeJ: str, nodes: list, inner=False):
    minDist = []
    lowest = ""

    if it.__eq__(0):
        dist = G.get_edge_data(start, nodeJ)["weight"]

    else:
        subDist = {}

        for node in nodes:
            subset = copy.deepcopy(nodes)
            subset.remove(node)

            dist1 = opf(it-1, node, subset, True)[0]
            dist2 = G.get_edge_data(node, nodeJ)["weight"]
            subDist[node] = dist1 + dist2

            if not inner:
                minDist.append(f'{dist1} + {dist2}')

        lowest, dist = min(subDist.items(), key=lambda x: x[1])

    return dist, lowest, minDist


In [7]:
seed = df.index.to_list()
seed.remove(start)

for i in seed:
    minDist, source, minSort = opf(0, str(i), [])
    print(f'f_0({i}, _) = {minDist}({start})')

for i in range(1, len(df) - 1):
    for j in seed:
        for k in list(it.combinations([item for item in seed if item not in j], i)):
            minDist, source, minSort = opf(i, str(j), list(k))
            print(f'f_{i}({j}, {list(k)}) = min{minSort} = {minDist}({source})')

minDist, source, minSort = opf(len(df) - 1, start, seed)
print(f'f_{len(df) - 1}({start}, {seed}) = min{minSort} = {minDist}({source})')


f_0(1, _) = 8(5)
f_0(2, _) = 6(5)
f_0(3, _) = 4(5)
f_0(4, _) = 2(5)
f_1(1, ['2']) = min['6 + 2'] = 8(2)
f_1(1, ['3']) = min['4 + 8'] = 12(3)
f_1(1, ['4']) = min['2 + 9'] = 11(4)
f_1(2, ['1']) = min['8 + 9'] = 17(1)
f_1(2, ['3']) = min['4 + 4'] = 8(3)
f_1(2, ['4']) = min['2 + 2'] = 4(4)
f_1(3, ['1']) = min['8 + 2'] = 10(1)
f_1(3, ['2']) = min['6 + 8'] = 14(2)
f_1(3, ['4']) = min['2 + 9'] = 11(4)
f_1(4, ['1']) = min['8 + 8'] = 16(1)
f_1(4, ['2']) = min['6 + 4'] = 10(2)
f_1(4, ['3']) = min['4 + 6'] = 10(3)
f_2(1, ['2', '3']) = min['8 + 2', '14 + 8'] = 10(2)
f_2(1, ['2', '4']) = min['4 + 2', '10 + 9'] = 6(2)
f_2(1, ['3', '4']) = min['11 + 8', '10 + 9'] = 19(3)
f_2(2, ['1', '3']) = min['12 + 9', '10 + 4'] = 14(3)
f_2(2, ['1', '4']) = min['11 + 9', '16 + 2'] = 18(4)
f_2(2, ['3', '4']) = min['11 + 4', '10 + 2'] = 12(4)
f_2(3, ['1', '2']) = min['8 + 2', '17 + 8'] = 10(1)
f_2(3, ['1', '4']) = min['11 + 2', '16 + 9'] = 13(1)
f_2(3, ['2', '4']) = min['4 + 8', '10 + 9'] = 12(2)
f_2(4, ['1', '2']) 

### Shortest tour


In [8]:
path = nx.algorithms.approximation.threshold_accepting_tsp(G, "greedy", source=start)
path


['5', '4', '2', '1', '3', '5']

### Graph by edge list


In [9]:
for edge in G.edges(data=True):
    print(f'{edge[0]} {edge[1]} {edge[-1]["weight"]}')


1 2 9
1 3 2
1 4 8
1 5 4
2 1 2
2 3 8
2 4 4
2 5 9
3 1 8
3 2 4
3 4 6
3 5 2
4 1 9
4 2 2
4 3 9
4 5 9
5 1 8
5 2 6
5 3 4
5 4 2
